In [2]:
import copy, torch, torch.nn as nn, torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import snntorch as snn
from snntorch import spikegen
from tqdm import tqdm

**parameters**

In [3]:
num_in, num_hid, num_out = 784, 256, 10
beta, T, bs, epochs, lr = 0.9, 50, 128, 3, 1e-3

**device**

In [4]:
device = torch.device("cuda")
torch.manual_seed(0)
device

device(type='cuda')

**arch network**

In [5]:

class SNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1  = nn.Linear(num_in , num_hid, bias=False)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2  = nn.Linear(num_hid, num_out, bias=False)
        self.lif2 = snn.Leaky(beta=beta)
    def forward(self, x):
        x = x.to(self.fc1.weight.dtype)
        mem1 = self.lif1.init_leaky().to(device=x.device, dtype=x.dtype)
        mem2 = self.lif2.init_leaky().to(device=x.device, dtype=x.dtype)
        out  = []
        for t in range(x.size(0)):
            spk1, mem1 = self.lif1(self.fc1(x[t]), mem1)
            spk1 = spk1.to(dtype=self.fc2.weight.dtype)        #
            spk2, mem2 = self.lif2(self.fc2(spk1), mem2)
            out.append(spk2)
        return torch.stack(out)                                # [T,B,10]

In [6]:
tr = transforms.Compose([transforms.ToTensor()])
train_ds = torchvision.datasets.MNIST("data", True , download=True, transform=tr)
test_ds  = torchvision.datasets.MNIST("data", False, download=True, transform=tr)
train_ld = DataLoader(train_ds, bs, True , num_workers=2, pin_memory=True)
test_ld  = DataLoader(test_ds , bs, False, num_workers=2, pin_memory=True)

In [7]:
net = SNN().to(device)
opt = torch.optim.Adam(net.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss().to(device)

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr=0.001)

**train phase**

In [11]:
def train(net,train_ld,epochs):
    for ep in range(1, epochs + 1):
        net.train()
        for imgs, lbls in tqdm(train_ld, desc=f"train {ep}/{epochs}"):
            imgs = imgs.to(device).view(imgs.size(0), -1) 
            lbls = lbls.to(device)
            spk  = spikegen.rate(imgs, T).to(device)
            loss = loss_fn(net(spk).sum(0), lbls)
            opt.zero_grad(); loss.backward(); opt.step()

In [12]:
train(net,train_ld,epochs)

train 3/3: 100%|██████████| 469/469 [00:37<00:00, 12.38it/s]


**test phase**

In [15]:
def test(model,loader):
    @torch.no_grad()
    def evaluate(model, loader, dtype):
        model.eval()
        correct = total = 0
        for imgs, lbls in loader:
            imgs = (imgs.to(device).view(imgs.size(0), -1) ).to(dtype)
            spk  = spikegen.rate(imgs, T).to(dtype).to(device)
            preds = model(spk).sum(0).argmax(1)
            correct += (preds == lbls.to(device)).sum().item()
            total   += lbls.size(0)
        return 100 * correct / total


    acc32 = evaluate(net, test_ld, torch.float32)


    net_fp16 = SNN().to(device)
    net_fp16.load_state_dict(net.state_dict())
    net_fp16.half().eval()
    acc16 = evaluate(net_fp16, test_ld, torch.float16)

    print(f"Accuracy FP32 : {acc32:.2f}%")
    print(f"Accuracy FP16 : {acc16:.2f}%")

In [16]:
test(net,test_ld)

Accuracy FP32 : 97.48%
Accuracy FP16 : 97.44%


In [17]:
# torch.save(net.state_dict(),'model_weights.pth')

In [18]:
# net.load_state_dict(torch.load('model_weights.pth'))